In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('C:\\Users\\koriv\\Desktop\\MachineLearning_DataScience\\Hands_On_Machine_Learning\\missing_value_handeling\\income_evaluation.csv', na_values = ' ?')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
#Check for ordinal and nominal columns
df[' education'].unique() #Is a ordinal column

array([' Bachelors', ' HS-grad', ' 11th', ' Masters', ' 9th',
       ' Some-college', ' Assoc-acdm', ' Assoc-voc', ' 7th-8th',
       ' Doctorate', ' Prof-school', ' 5th-6th', ' 10th', ' 1st-4th',
       ' Preschool', ' 12th'], dtype=object)

In [4]:
#check for total number of missing values in the dataset
df.isna().sum()

age                   0
 workclass         1836
 fnlwgt               0
 education            0
 education-num        0
 marital-status       0
 occupation        1843
 relationship         0
 race                 0
 sex                  0
 capital-gain         0
 capital-loss         0
 hours-per-week       0
 native-country     583
 income               0
dtype: int64

In [5]:
# As observered all the missing values were in catogorical side and numerical columns seems to be full.
# Lets add some missing values into the numberical columns as well, to see how we can better handel them
dft= df
# add missing values to hours per week 
np.random.seed(seed=0)
h = np.random.choice(a=df.index, replace=False, size=20)
dft.loc[h, ' hours-per-week'] = np.nan
# add missing values to age 
np.random.seed(seed=10)
a = np.random.choice(a=df.index, replace=False, size=28)
dft.loc[a, 'age'] = np.nan

In [6]:
#check for total number of missing values in the new transformed dataset
dft.isna().sum()

age                  28
 workclass         1836
 fnlwgt               0
 education            0
 education-num        0
 marital-status       0
 occupation        1843
 relationship         0
 race                 0
 sex                  0
 capital-gain         0
 capital-loss         0
 hours-per-week      20
 native-country     583
 income               0
dtype: int64

In [7]:
# Split the data into training and test dataset to check generalization error before imputation

X_train, X_test, y_train, y_test = train_test_split(dft.drop(' income', axis=1),
                                                    dft[' income'], test_size=0.2,
                                                    random_state=5)

In [17]:
# KNN can only handel number so all catogorical(nominal and ordinal) should be converted to numbers via encoding
# Check columns datatype
print("age dataype: ", X_train['age'].dtypes)
print(" education dataype: ", X_train[' education'].dtypes)
num = [col for col in X_train.columns if X_train[col].dtypes != 'O']
print("numerical columnsare: ",num)
X_train[num].head()

age dataype:  float64
 education dataype:  object
numerical columnsare:  ['age', ' fnlwgt', ' education-num', ' capital-gain', ' capital-loss', ' hours-per-week']


,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
21425,55.0,238216,9,0,0,40.0
28707,24.0,306460,9,0,0,40.0
4455,48.0,213140,4,0,0,40.0
2231,36.0,127306,13,0,0,40.0
18864,53.0,103586,13,0,0,55.0


In [20]:
#Fit and transform KNNImputer with Training Data
from sklearn.impute import KNNImputer
knn = KNNImputer(n_neighbors=5, add_indicator=True)
knn.fit(X_train[num]) # Using a dataset with allnumerical values
pd.DataFrame(knn.transform(X_train[num])).head()

,0,1,2,3,4,5,6,7
0,55.0,238216.0,9.0,0.0,0.0,40.0,0.0,0.0
1,24.0,306460.0,9.0,0.0,0.0,40.0,0.0,0.0
2,48.0,213140.0,4.0,0.0,0.0,40.0,0.0,0.0
3,36.0,127306.0,13.0,0.0,0.0,40.0,0.0,0.0
4,53.0,103586.0,13.0,0.0,0.0,55.0,0.0,0.0


In [23]:
# Propagating imputed object to the testing set.
print("null values in the Test data: \n", X_test[num].isna().sum())
pd.DataFrame(knn.transform(X_test[num])).isna().sum().sum()


null values in the Test data: 
 age                5
 fnlwgt            0
 education-num     0
 capital-gain      0
 capital-loss      0
 hours-per-week    1
dtype: int64


0

## Each value missing from the Test Data is computed from the simiral neighbors in the Training Data and not from the Data in the Test DataSet.